# Neural network models

In [7]:
import torch
import torch.nn as nn
import math
from torchvision import models
from collections import OrderedDict

## U-Net Network
num_classes = Number of classes expecteed from output

In [8]:
def conv2d_layer(channels_in, channels_out, kernel_size, padding, stride, dilation):
    layer = nn.ModuleList()
  
    layer.append(nn.Conv2d(
        in_channels = channels_in, 
        out_channels = channels_out, 
        kernel_size = kernel_size, 
        stride = stride, 
        padding = padding, 
        dilation = dilation
        )
    )
    layer.append(nn.ReLU())

    return layer

In [9]:
class Encoder_Block(nn.Module):
    def __init__(self, channels_in, channels_out, params: dict):
        super().__init__()

        # Config parameters
        try:
            block_width = params['block_width']
            kernel_size = params['kernel_size']
            padding = params['padding']
            stride = params['stride']
            dilation = params['dilation']
            pool_type = params['pool_type']
            pool_size = params['pool_kernel_size']
        except KeyError as e:
            raise Exception(f'Parameter "{e.args[0]}" NOT found!')
        
        # Add first Conv2D layer
        self.layers = nn.ModuleList()
        self.layers.extend(conv2d_layer(channels_in, channels_out, kernel_size, padding, stride, dilation))

        # Add other Conv2D layers
        for _ in range(block_width - 1):
            self.layers.extend(conv2d_layer(channels_out, channels_out, kernel_size, padding, stride, dilation))

        # Add pooling layer
        match pool_type:
            case 'max':
                self.pooling = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
            case 'avg':
                self.pooling = nn.AvgPool2d(kernel_size=pool_size, stride=pool_size)


    def forward_noSkip(self, x):
        # Convolution
        for layer in self.layers:
            x = layer(x)

        # Pooling
        pooled = self.pooling(x)

        return pooled
    
    def forward_skip(self, x):
        # Convolution
        for layer in self.layers:
            x = layer(x)

        # Pooling
        pooled = self.pooling(x)

        return (pooled, x)

In [10]:
class Decoder_Block(nn.Module):
    def __init__(self, channels_in, channels_out, padding_t, params: dict):
        super().__init__()

        # Config parameters
        try:
            block_width = params['block_width']
            kernel_size = params['kernel_size']
            padding = params['padding']
            stride = params['stride']
            dilation = params['dilation']
            pool_size = params['pool_kernel_size']
            skip_features = params['skip_features']
        except KeyError as e:
            raise Exception(f'Parameter "{e.args[0]}" NOT found!')

        # Add transpose convolution
        self.convT = nn.ConvTranspose2d (
            in_channels = channels_in, 
            out_channels = math.floor(channels_in / 2), 
            kernel_size = pool_size, 
            stride = pool_size, 
            output_padding = padding_t
        )

        match skip_features:
            case "concat":
                channels_in = math.floor(channels_in / 2)
                pass
            case "none":
                pass
            case "sdi":
                pass


        # Add first Conv2D layer
        self.layers = nn.ModuleList()
        self.layers.extend(conv2d_layer(channels_in, channels_out, kernel_size, padding, stride, dilation))

        # Add other Conv2D layers
        for _ in range(block_width - 1):
            self.layers.extend(conv2d_layer(channels_out, channels_out, kernel_size, padding, stride, dilation))

    def forward_noSkip(self, x):
        # Transposed convolution
        x = self.convT(x)

        # Convolution
        for layer in self.layers:
            x = layer(x)

        return x

    def forward_skip(self, x, enc_feature):
        # Transposed convolution
        x = self.convT(x)

        # Copy and crop features
        w = x.size(-1)
        c = math.floor((enc_feature.size(-1) - w) / 2)

        print(x.size())   
        print(enc_feature.size())
        #enc_features = enc_features[:, :, c:c+w , c:c+w]
        x = torch.cat([enc_feature, x], dim = 1)

        # Convolution
        for layer in self.layers:
            x = layer(x)

        return x

In [11]:
class U_Net(nn.Module):
    def __init__(self, channels_in, num_classes, params: dict):
        super().__init__()

        # Config parameters
        try:
            channel_mul = params['channel_mul']
            depth = params['network_depth']
            channels_out = params['channels_out_init']
            padding_convT = params['padding_convT']
            self.skip_features = params['skip_features']
        except KeyError as e:
            raise Exception(f'Parameter "{e.args[0]}" NOT found!')
        
        # Create encoder
        self.encoders = nn.ModuleList()

        for _ in range(depth):
            self.encoders.append(Encoder_Block(channels_in, channels_out, params))
            channels_in = channels_out
            channels_out = math.floor(channels_out * channel_mul)

        # Create bridge as Conv2D layer
        last_encoder = Encoder_Block(channels_in, channels_out, params)
        self.bridge = last_encoder.layers[0]

        channels_in = channels_out
        channels_out = math.floor(channels_in / channel_mul)

        # Create decoder
        self.decoders = nn.ModuleList()
        
        for _ in range(depth) :
            paddingT = padding_convT.pop(0)
            self.decoders.append(Decoder_Block(channels_in, channels_out, paddingT, params))
            channels_in = channels_out
            channels_out = math.floor(channels_out / channel_mul)

        # Create output layer (1x1 convolution, return logits)
        self.output_layer = nn.Conv2d(
            in_channels = channels_in, 
            out_channels = num_classes, 
            kernel_size = 1
        )
        

    def forward(self, x):
        match self.skip_features:
            # No skipping
            case "none":
                print("This U-Net does NOT skip any connections!")
                # Encoding
                for enc in self.encoders:
                    x = enc.forward_noSkip(x)

                # Bridge between encoder and decoder
                x = self.bridge(x)

                # Decoding
                for dec in self.decoders:
                    x = dec.forward_noSkip(x)

            # Cat tensors => features + pooled output
            case "concat":
                print("This U-Net skips connections using concatenation!")
                # Features created during enconding
                features = []

                # Encoding
                for enc in self.encoders:
                    x, enc_feature = enc.forward_skip(x)
                    features.append(enc_feature)

                # Bridge between encoder and decoder
                x = self.bridge(x)

                # Decoding
                for dec in self.decoders:
                    enc_output = features.pop()
                    x = dec.forward_skip(x, enc_output)

            case "sdi":
                pass
    

        # Output as logits
        x = self.output_layer(x)

        return x

In [12]:
def conv2d_output_shape(height, width, conv2d: nn.Module):
    height = ((height + (2 * conv2d.padding[0]) - (conv2d.dilation[0] * (conv2d.kernel_size[0] - 1)) - 1) / conv2d.stride[0]) + 1
    height = math.floor(height)
    
    width = ((width + (2 * conv2d.padding[1]) - (conv2d.dilation[1] * (conv2d.kernel_size[1] - 1)) - 1) / conv2d.stride[1]) + 1
    width = math.floor(width)

    return (height, width, conv2d.out_channels)

def pool_output_shape(height, width, conv2d: nn.Module):
    height = ((height + (2 * conv2d.padding) - (conv2d.dilation * (conv2d.kernel_size - 1)) - 1) / conv2d.stride) + 1
    height = math.floor(height)
    
    width = ((width + (2 * conv2d.padding) - (conv2d.dilation * (conv2d.kernel_size - 1)) - 1) / conv2d.stride) + 1
    width = math.floor(width)

    return (height, width)

def convT_output_shape(height, width, convT: nn.Module):
    height = (height - 1) * convT.stride[0] - (2 * convT.padding[0]) + (convT.dilation[0] * (convT.kernel_size[0] - 1)) + convT.output_padding[0] + 1
    
    width = (width - 1) * convT.stride[1] - (2 * convT.padding[1]) + (convT.dilation[1] * (convT.kernel_size[1] - 1)) + convT.output_padding[1] + 1
    
    return (height, width, convT.out_channels)


def output_shapes(net : U_Net, image):
    h = image.size(-2)  # height
    w = image.size(-1)  # width
    depth = len(net.encoders) + 1

    print("----------------------------------------------")
    print(f"CNN with depth = {depth}")
    print("----------------------------------------------")

    i = 1
    for enc in net.encoders:
        print(f"||Layer {i} - encoders||")

        for l in enc.layers:
            if not isinstance(l, nn.ReLU):
                (h,w,c) = conv2d_output_shape(h, w, l)
                print (f"Conv2D -- H = {h} | W = {w} | C = {c}")

        (h,w) = pool_output_shape(h, w, enc.pooling)
        print (f"Pooling -- H = {h} | W = {w} | C = {c}") 

        print()
        i += 1
            
    print(f"||Layer {i} - bridge||")
    (h,w,c) = conv2d_output_shape(h, w, net.bridge)
    print (f"H = {h} | W = {w} | C = {c}")
    print("----------------------------------------------")

    for dec in net.decoders:
        print(f"||Layer {i} - decoders||")

        (h,w,c) = convT_output_shape(h, w, dec.convT)
        print (f"Transpose -- H = {h} | W = {w} | C = {c}")   

        for l in dec.layers:
            if not isinstance(l, nn.ReLU):
                (h,w,c) = conv2d_output_shape(h, w, l)
                print (f"Conv2D -- H = {h} | W = {w} | C = {c}")   

        print()
        i -= 1


    print(f"||Layer {i} - output||")
    (h,w,c) = conv2d_output_shape(h, w, net.output_layer)
    print (f"H = {h} | W = {w} | C = {c}")
    print("----------------------------------------------")

In [15]:
from sklearn.model_selection import train_test_split

%run config.ipynb

x = torch.load('tensor_images.pt')
y = torch.load('tensor_labels.pt')

print(x.shape)
print(y.shape)

print(x.dtype)
print(y.dtype)


print(y)



#net = U_Net(2, 10, config_to_dict(config_unet))

#output_shapes(net, x)

#out = net(x)

torch.Size([238, 3, 500, 500])
torch.Size([238, 3, 500, 500])
torch.float64
torch.uint8
tensor([[[[208, 219, 175,  ..., 201, 221, 231],
          [214, 226, 167,  ..., 195, 215, 213],
          [186, 197, 154,  ..., 186, 192, 198],
          ...,
          [200, 192, 204,  ..., 207, 221, 234],
          [199, 202, 210,  ..., 234, 246, 245],
          [209, 209, 212,  ..., 252, 245, 223]],

         [[183, 195, 149,  ..., 145, 170, 180],
          [189, 202, 141,  ..., 144, 166, 165],
          [160, 172, 128,  ..., 141, 149, 157],
          ...,
          [155, 146, 152,  ..., 185, 201, 215],
          [152, 154, 159,  ..., 216, 229, 228],
          [168, 169, 172,  ..., 238, 230, 207]],

         [[234, 239, 207,  ..., 198, 222, 234],
          [244, 246, 201,  ..., 199, 222, 223],
          [226, 231, 194,  ..., 205, 211, 216],
          ...,
          [219, 209, 215,  ..., 222, 233, 244],
          [212, 214, 219,  ..., 243, 252, 252],
          [217, 216, 219,  ..., 254, 252, 236]]